In [7]:
import pandas as pd
from datetime import datetime

# I checked the difference in size between csv and parquet and it is huuuuuge!
df = pd.read_csv('ultramarathon.csv', low_memory=False) #789.9 MB
# df = df.to_parquet('ultramarathon.parquet') #128,7 MB

In [9]:
#over 7 mln rows in 13 columns
df.shape

(7117634, 13)

In [10]:
#As we can see many of columns are objects where I would expect integers or floats so we need to investigate it

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7117634 entries, 0 to 7117633
Data columns (total 13 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Year of event              int64  
 1   Event dates                object 
 2   Event name                 object 
 3   Event distance/length      object 
 4   Event number of finishers  int64  
 5   Athlete performance        object 
 6   Athlete club               object 
 7   Athlete country            object 
 8   Athlete year of birth      float64
 9   Athlete gender             object 
 10  Athlete age category       object 
 11  Athlete average speed      object 
 12  Athlete ID                 int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 705.9+ MB


In [11]:
#That's definitely not the cleanest dataset ever

df.describe()

,Year of event,Event number of finishers,Athlete year of birth,Athlete ID
count,7.117634e+06,7.117634e+06,6.558280e+06,7.117634e+06
mean,2.012990e+03,1.310716e+03,1.970613e+03,5.275223e+05
std,9.074683e+00,2.973447e+03,1.264591e+01,4.634639e+05
min,1.798000e+03,0.000000e+00,1.193000e+03,0.000000e+00
25%,2.011000e+03,8.700000e+01,1.963000e+03,1.304470e+05
50%,2.016000e+03,2.270000e+02,1.972000e+03,3.775990e+05
75%,2.019000e+03,7.690000e+02,1.980000e+03,9.164148e+05
max,2.022000e+03,2.002700e+04,2.021000e+03,1.600755e+06


In [15]:
df.head()

,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
0,2018,06.01.2018,Selva Costera (CHI),50km,22,4:51:39 h,Tnfrc,CHI,1978.0,M,M35,10.286,0
1,2018,06.01.2018,Selva Costera (CHI),50km,22,5:15:45 h,Roberto Echeverría,CHI,1981.0,M,M35,9.501,1
2,2018,06.01.2018,Selva Costera (CHI),50km,22,5:16:44 h,Puro Trail Osorno,CHI,1987.0,M,M23,9.472,2
3,2018,06.01.2018,Selva Costera (CHI),50km,22,5:34:13 h,Columbia,ARG,1976.0,M,M40,8.976,3
4,2018,06.01.2018,Selva Costera (CHI),50km,22,5:54:14 h,Baguales Trail,CHI,1992.0,M,M23,8.469,4


In [ ]:
from datetime import timedelta

df = pd.read_parquet('ultramarathon.parquet')


df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)

# Quick fix here: daily I see tables where columns haves spaces instead of underscores 
# or other signs so using lambda to treat columns as 'x' and then replacing in every 'x' space for underscore is a gamechanger!


# I used data after 1999, wasn't sure about accuracy in Athlete performance in earlier years
df_filtered = df[df['Year_of_event'] >= 2000]


# Wanted to check if suffix are only km and miles. After using this code I saw that there are values like 'es'  because of word Stages in Distance column
# or 'en' for Etappen suffix
# I only have taken 'one day' events so any stagedd raced was filtered
df_filtered['length'] = df_filtered['Event_distance/length'].str[-2:] #this means i start from the end of the string and take 2 last characters
df_filtered['length'] = df_filtered['length'].astype('string')


df_filtered = df_filtered[~df_filtered['length'].str.contains('d|h|en|p.')] # Here and below are weird values i needed to get rid of.
# Here are 2 last characters
df_filtered = df_filtered[~df_filtered['Event_distance/length'].str.contains('stages|6:40|07:35|tapp')]

# Created this dictionary to use it later in astype function
dict_for_formats = {'Event_dates': 'string', 'Event_distance/length': 'string', 
                    'Event_name': 'string', 'Athlete_club': 'string','Athlete_country':'string',
                    'Athlete_gender':'string', 'Athlete_age_category': 'string', 'Athlete_performance': 'string'}
df_filtered = df_filtered.astype(dict_for_formats)

# The .copy() method in pandas is used to create a deep copy 
# of a DataFrame or a Series. It ensures that any changes made to the copied DataFrame do not affect the original DataFrame.
df_filtered_2 = df_filtered.copy()

df_filtered_2 = df_filtered[['Year_of_event',
                            'Event_dates','Event_name', 'Event_distance/length','Athlete_performance',
                            'Athlete_country', 'Athlete_year_of_birth','Athlete_gender','Athlete_age_category', 'length']]


# I could also create dictionary for columns names but I decided to do it this way
df_filtered_2 = df_filtered_2.rename(columns={'Athlete_year_of_birth': 'Age', 
                                 'Year_of_event':'Year',
                                 'Event_dates':'Date',
                                 'Event_name':'Event',
                                 'Event_distance/length':'Distance',
                                 'Athlete_performance':'Time',
                                 'Athlete_country':'Country',
                                 'Athlete_gender':'Gender'})

# This is interesting. Because some event's date where like 08-10.12.2020 I needed to get rid of '-' and everything before
df_filtered_2['Date'] = df_filtered_2['Date'].str.replace('.*.-', '', regex=True)

# By using errors='coerce', you are essentially telling the pd.to_datetime() function 
# to handle any problematic date values gracefully by converting them to NaT instead of raising an error and stopping the execution of the code.
df_filtered_2 = df_filtered_2.copy()
df_filtered_2['Date'] = pd.to_datetime(df_filtered_2['Date'], format='%d.%m.%Y', errors='coerce')

# Cleaning Distance column from unnecessary characters
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.rstrip()
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.lower()


df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(',','.')

# Breaking down what happend in str.replace():
# r'(\d+)\s?mile\b': This is the regular expression pattern that matches a number followed by the word "mile":
# r before the string indicates that it is a raw string, which means backslashes are treated as literal characters.
# (\d+) is a capturing group that matches one or more digits. The parentheses capture the matched digits for later use.
# \s? matches an optional whitespace character.
# mile matches the literal word "mile".
# \b is a word boundary that ensures the match is at the end of a word.

# r'\1 mi': 
# \1 refers to the first capturing group in the regular expression pattern, which is the matched number of miles.
# mi is the literal string "mi" that replaces the matched pattern.

# After many checking of values in Distance column and checking uniques in length column 
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(r'(\d+)\s?mile\b', r'\1 mi', regex=True)
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(r'(\d+)\s?k\b', r'\1 km', regex=True)
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(r'(\d+)\s?m\b', r'\1 mi', regex=True)
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(r'(\d+)\s?miles\b', r'\1 mi', regex=True)
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace('+','')
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(r'(\d+)\s?mi\b', r'\1 mi', regex=True)
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace('100km split','100km')
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace('4x52','208')
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace('112.km','112')

# I did it again to check if there are any other values that I need to get rid of
df_filtered_2['length'] = df_filtered_2['Distance'].str[-2:]
df_filtered_2['length'] = df_filtered_2['length'].astype('string')

# Here I started to clean Time and Distance columns to get only numerical values
df_filtered_2['Time'] = df_filtered_2['Time'].str.replace(r'(\d+)\s?h\b', r'\1', regex=True)
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(r'(\d+)\s?mi\b', r'\1', regex=True)
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace(r'(\d+)\s?km\b', r'\1', regex=True)

# I found couple rows with '229.100.100' value so changed it for 229
df_filtered_2['Distance'] = df_filtered_2['Distance'].str.replace('229.100.100','229')
# Wasnt sure if 00:00:00 was no time recorded or DNFs so I decided to get rid of it
df_filtered_2 = df_filtered_2[~df_filtered_2['Time'].str.contains('0:00:00')]

# I know that even for PRO athletes some races are undoable to finish < 24h but didn't want to transform data again 
df_filtered_2 = df_filtered_2[~df_filtered_2['Time'].str.contains('d')]

df_filtered_2['Time'] = pd.to_datetime(df_filtered_2['Time'], format='%H:%M:%S', errors='coerce').dt.time

df_filtered_2['Distance'] = df_filtered_2['Distance'].astype(float)

df_filtered_2['Distance'] = df_filtered_2['Distance'].round(0)

df_filtered_2['Distance'] = df_filtered_2['Distance'].astype(int)

df_filtered_2 = df_filtered_2[df_filtered_2['Time'].notna()]

# Defined here a function that multiplied ditance with 1.6 if it was in miles (length column == mi)
def num_distance(row):
    if row['length'] == 'mi':
        return row['Distance'] * 1.6
    else:
        return row['Distance']

# Apply this function
df_filtered_2['Distance_in_km'] = df_filtered_2.apply(num_distance, axis=1)

df_filtered_2['Time_2'] = df_filtered_2['Time'].astype('string')

# Function to convert time format
def convert_time(time_str):
        try:
            # Parse time and reformat it
            return datetime.strptime(time_str, '%H:%M:%S').strftime('%H:%M:%S')
        except ValueError:
            try:
                # If the time is missing the hour part, add it
                return datetime.strptime(time_str, '%M:%S').strftime('00:%M:%S')
            except ValueError:
                # Handle other formats or return the original string
                return time_str

# Applying the function to the column
df_filtered_2['Time_2'] = df_filtered_2['Time_2'].apply(convert_time)

# Converting 'Time' column to timedelta type
df_filtered_2['Time_2'] = df_filtered_2['Time_2'].astype('timedelta64[ns]')

# Converting 'Athlete_performance' from timedelta to total minutes
df_filtered_2['Total_minutes'] = df_filtered_2['Time_2'].dt.total_seconds() / 60

# Calculating pace as minutes per kilometer
df_filtered_2['Pace_min_per_km'] = df_filtered_2['Total_minutes'] / df_filtered_2['Distance']
df_filtered_2['Pace_min_per_km']  = df_filtered_2['Pace_min_per_km'].round(2)

# Here I didn't have a good idea how to convert 4.5 to 4:30 so I made these modification to get 4.30. 
# Decided to do it in that way because I wanted to have ability to for example sort it
df_filtered_2['Pace_min_per_km'] = df_filtered_2['Pace_min_per_km'].astype('string')
df_filtered_2['min_to_conv'] = df_filtered_2['Pace_min_per_km'].apply(lambda x: x.split('.')[1])
df_filtered_2['min_to_conv'] = '0.'+ df_filtered_2['min_to_conv']
df_filtered_2['min_to_conv'] = df_filtered_2['min_to_conv'].astype(float)*60
df_filtered_2['min_to_conv'] = df_filtered_2['min_to_conv'].astype('string')
df_filtered_2['min_to_conv'] = df_filtered_2['min_to_conv'].str.replace('.','')
df_filtered_2['Pace'] = df_filtered_2['Pace_min_per_km'].apply(lambda x: x.split('.')[0]) + '.' + df_filtered_2['min_to_conv'].astype('string')
df_filtered_2['Pace'] = df_filtered_2['Pace'].astype(float).round(2)

# World record in Ultramarathon is something like 3:40 so everything below 4:00 so a little bit weird for me
# > 10:00 is walk pace so no need to keep it here 
df_filtered_2 = df_filtered_2[(df_filtered_2['Pace'] >= 4.00) & (df_filtered_2['Pace'] <= 10.00)]

# Of course ultramarathon is a distance bigger than marathon so filtering below
df_filtered_2 = df_filtered_2[(df_filtered_2['Distance_in_km'] >= 42.195)]

# df_filtered_2['Age'] = df_filtered_2['Year'] - df_filtered_2['Age']

# df_filtered_2 = df_filtered_2[(df_filtered_2['Age'] >= 18) & (df_filtered_2['Age'] <= 60)]

# df_filtered_2['Age'] = df_filtered_2['Age'].fillna(df_filtered_2['Age'].mean())

# df_filtered_2['Age'] = df_filtered_2['Age'].astype('int16')

df_filtered_2['Year'] = df_filtered_2['Year'].astype('int16')
df_filtered_2 = df_filtered_2.drop(columns=['Pace_min_per_km', 'min_to_conv', 'Total_minutes', 'Time_2', 'length', 'Distance', 'Age'])

df_filtered_2['Athlete_age_category'] = df_filtered_2['Athlete_age_category'].astype('category')
df_filtered_2['Gender'] = df_filtered_2['Gender'].astype('category')
df_filtered_2['Event'] = df_filtered_2['Event'].astype('category')
df_filtered_2['Country'] = df_filtered_2['Country'].astype('category')

df_filtered_2.reset_index(drop=True, inplace=True)

#df_filtered_2.to_parquet('ultramarathons_clean.parquet', index=False)


/var/folders/z0/qrctxq5d6f51khdbp1n6z0zh0000gn/T/ipykernel_66459/2457026069.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['length'] = df_filtered['Event_distance/length'].str[-2:]
/var/folders/z0/qrctxq5d6f51khdbp1n6z0zh0000gn/T/ipykernel_66459/2457026069.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['length'] = df_filtered['length'].astype('string')


In [2]:
dir(str)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'stri

In [9]:
import pandas as pd
df_string = pd.DataFrame(['capitalize',
 'casefold',
 'center',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isupper',
 'join',
 'ljust',
 'lower',
 'lstrip',
 'maketrans',
 'partition',
 'removeprefix',
 'removesuffix',
 'replace',
 'rfind',
 'rindex',
 'rjust',
 'rpartition',
 'rsplit',
 'rstrip',
 'split',
 'splitlines',
 'startswith',
 'strip',
 'swapcase',
 'title',
 'translate',
 'upper',
 'zfill'])

method_descriptions = {
    method: getattr(str, method).__doc__.strip().split('\n')[0]
    for method in df[0]
}



In [10]:
method_descriptions

{'capitalize': 'Return a capitalized version of the string.',
 'casefold': 'Return a version of the string suitable for caseless comparisons.',
 'center': 'Return a centered string of length width.',
 'count': 'S.count(sub[, start[, end]]) -> int',
 'encode': 'Encode the string using the codec registered for encoding.',
 'endswith': 'S.endswith(suffix[, start[, end]]) -> bool',
 'expandtabs': 'Return a copy where all tab characters are expanded using spaces.',
 'find': 'S.find(sub[, start[, end]]) -> int',
 'format': 'S.format(*args, **kwargs) -> str',
 'format_map': 'S.format_map(mapping) -> str',
 'index': 'S.index(sub[, start[, end]]) -> int',
 'isalnum': 'Return True if the string is an alpha-numeric string, False otherwise.',
 'isalpha': 'Return True if the string is an alphabetic string, False otherwise.',
 'isascii': 'Return True if all characters in the string are ASCII, False otherwise.',
 'isdecimal': 'Return True if the string is a decimal string, False otherwise.',
 'isdigi